This notebook is based on the Jonathan K. Kummerfeld's work. And this notebook runs the coreferecne resolution model described in "SpanBERT: Improving Pre-training by Representing and Predicting Spans" by Mandar Joshi, Danqi Chen, Yinhan Liu, Daniel S. Weld, Luke Zettlemoyer, Omer Levy, and released here: https://github.com/mandarjoshi90/coref

#Configuration
load data

In [ ]:
import json

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
def get_json_list(multiclass_file, software = 'doccano'):
    with open(multiclass_file, 'r') as json_file:
        json_list = list(json_file)

    annotations_json = []
    for line in json_list:
        annotations_json.append(json.loads(line))

    if software == 'prodigy':
        annotations_json = prodigy_to_doccano(annotations_json)

    return annotations_json

In [ ]:
file = '/content/drive/MyDrive/ieee_ner_coref/assets/data/annotations/yx_converted_abs_combine_114.jsonl'
annotations_json = get_json_list(file, software = 'doccano')

In [ ]:
text = []
for i in range(len(annotations_json)):
  text.append(annotations_json[i]['text'])

In [ ]:
with open("span_text.txt", 'w') as f:
        for i in text:
            f.write(i + '\n')

In [ ]:
data = []
with open("/content/span_text.txt", "r") as f:
  for line in f.readlines():
    line = line.strip('\n')
    data.append(line)

In [ ]:
#Using this function if the text is too long
def split_text(text):
  n_ = int(len(text)/2)
  idx = n_
  for i,char in enumerate (text[n_:]):
    if char == '.':
      idx +=i
      p1 = text[:idx]
      p2 = text[idx+1:]
      break
  return p1, p2


In [ ]:
text = data.copy()

In [ ]:
text = [
"Firefly is an American space Western drama television series which ran from 2002-2003, created by writer and director Joss Whedon, under his Mutant Enemy Productions label.",
"Whedon served as an executive producer, along with Tim Minear.",
"The series is set in the year 2517, after the arrival of humans in a new star system and follows the adventures of the renegade crew of Serenity, a 'Firefly-class' spaceship.",
"The ensemble cast portrays the nine characters who live on Serenity.",
"Whedon pitched the show as 'nine people looking into the blackness of space and seeing nine different things.'",
"The show explores the lives of a group of people, some of whom fought on the losing side of a civil war, who make a living on the fringes of society as part of the pioneer culture of their star system.",
"In this future, the only two surviving superpowers, the United States and China, fused to form the central federal government, called the Alliance, resulting in the fusion of the two cultures.",
"According to Whedon's vision, 'nothing will change in the future: technology will advance, but we will still have the same political, moral, and ethical problems as today.'",
"Firefly premiered in the U.S. on the Fox network on September 20, 2002.",
"By mid-December, Firefly had averaged 4.7 million viewers per episode and was 98th in Nielsen ratings.",
"It was canceled after 11 of the 14 produced episodes were aired.",
"Despite the relatively short life span of the series, it received strong sales when it was released on DVD and has large fan support campaigns.",
"It won a Primetime Emmy Award in 2003 for Outstanding Special Visual Effects for a Series.",
"TV Guide ranked the series at No. 5 on their 2013 list of 60 shows that were 'Cancelled Too Soon.'",
"The post-airing success of the show led Whedon and Universal Pictures to produce Serenity, a 2005 film which continues from the story of the series, and the Firefly franchise expanded to other media, including comics and a role-playing game.",
]

In [ ]:
text[0]

'This paper shows how multiple aspects of highest resolution airborne InSAR data can be combined by simultaneous backward geocoding of the individual aspects. Due to slant range geometry and effects like layover and shadowing, fusing multi-aspect SAR data of urban areas is not possible in image space. Therefore the fusion process is modeled via object space, enabling the exploitation of redundant observations for each height value.'

In [ ]:
text1 = []
for i in range(len(text)):
  if len(text[i])<900:
    text1.append(text[i])

In [ ]:
text1 = []

In [ ]:
part1,part2 = split_text(text[15])

In [ ]:
text1.append(part1)
text1.append(part2)

In [ ]:
text = text1.copy()

specify the fine-tune data type and model:

In [ ]:
genre = "nw"
# The Ontonotes data for training the model contains text from several sources
# of very different styles. You need to specify the most suitable one out of:
# "bc": broadcast conversation
# "bn": broadcast news
# "mz": magazine
# "nw": newswire
# "pt": Bible text
# "tc": telephone conversation
# "wb": web data

model_name = "spanbert_base"
# The fine-tuned model to use. Options are:
# bert_base
# spanbert_base
# bert_large
# spanbert_large

#Installation the repo and requirements

In [ ]:
! git clone https://github.com/mandarjoshi90/coref.git
%cd coref

Cloning into 'coref'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 734 (delta 2), reused 0 (delta 0), pack-reused 728
Receiving objects: 100% (734/734), 4.17 MiB | 18.59 MiB/s, done.
Resolving deltas: 100% (441/441), done.
/content/coref


In [ ]:
! sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/; s/scikit-learn==0.19.1/scikit-learn==0.21/; s/scipy==1.0.0/scipy==1.6.2/' < requirements.txt > tmp
! mv tmp requirements.txt

! sed 's/.D.GLIBCXX.USE.CXX11.ABI.0//' < setup_all.sh  > tmp
! mv tmp setup_all.sh 
! chmod u+x setup_all.sh 

Set some environment variables. The data directory one is used by the system, the other is so we can use the model defined above.

In [ ]:
import os
os.environ['data_dir'] = "."
os.environ['CHOSEN_MODEL'] = model_name

In [ ]:
%tensorflow_version 2.x
! pip uninstall -y tensorflow
! pip install -r requirements.txt --log install-log.txt -q
! ./setup_all.sh

Found existing installation: tensorflow 2.5.0
Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 99 kB 5.4 MB/s 
     |████████████████████████████████| 1.2 MB 30.6 MB/s 
     |████████████████████████████████| 157 kB 72.6 MB/s 
     |████████████████████████████████| 6.6 MB 24.9 MB/s 
     |████████████████████████████████| 543 kB 43.8 MB/s 
     |████████████████████████████████| 52 kB 899 kB/s 
     |████████████████████████████████| 2.2 MB 52.1 MB/s 
     |████████████████████████████████| 264 kB 74.6 MB/s 
     |████████████████████████████████| 889 kB 45.0 MB/s 
     |████████████████████████████████| 126 kB 67.1 MB/s 
     |████████████████████████████████| 147 kB 57.0 MB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 41 kB 795 kB/s 
     |████████████████████████████████| 87 kB 8.6 MB/s 
     |████████████████████████████████| 20.3 MB 1.5 MB/s 
     |█████████████

get fine-tune model

In [ ]:
! ./download_pretrained.sh $CHOSEN_MODEL

--2021-08-02 12:04:20--  http://nlp.cs.washington.edu/pair2vec/spanbert_base.tar.gz
Resolving nlp.cs.washington.edu (nlp.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to nlp.cs.washington.edu (nlp.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1633726311 (1.5G) [application/x-gzip]
Saving to: ‘./spanbert_base.tar.gz’

spanbert_base.tar.g 100%[===================>]   1.52G  99.5MB/s    in 23s     

2021-08-02 12:04:43 (67.3 MB/s) - ‘./spanbert_base.tar.gz’ saved [1633726311/1633726311]

spanbert_base/
spanbert_base/checkpoint
spanbert_base/model.max.ckpt.index
spanbert_base/stdout.log
spanbert_base/bert_config.json
spanbert_base/vocab.txt
spanbert_base/model.max.ckpt.data-00000-of-00001
spanbert_base/events.out.tfevents.1561596094.learnfair1413


# Data Preparation and Prediction

Process the data to be in the required input format.

In [ ]:
from bert import tokenization
import json

def data_convert(text):
  data = {
      'doc_key': genre,
      'sentences': [["[CLS]"]],
      'speakers': [["[SPL]"]],
      'clusters': [],
      'sentence_map': [0],
      'subtoken_map': [0],
  }

  # Determine Max Segment
  max_segment = None
  for line in open('experiments.conf'):
      if line.startswith(model_name):
          max_segment = True
      elif line.strip().startswith("max_segment_len"):
          if max_segment:
              max_segment = int(line.strip().split()[-1])
              break

  tokenizer = tokenization.FullTokenizer(vocab_file="cased_config_vocab/vocab.txt", do_lower_case=False)
  subtoken_num = 0
  sent_n  = 0
  for sent_num, line in enumerate(text):
      raw_tokens = line.split()
      tokens = tokenizer.tokenize(line)
      if len(tokens) + len(data['sentences'][-1]) >= max_segment:
          data['sentences'][-1].append("[SEP]")
          data['sentences'].append(["[CLS]"])
          data['speakers'][-1].append("[SPL]")
          data['speakers'].append(["[SPL]"])
          data['sentence_map'].append(sent_num - 1)
          data['subtoken_map'].append(subtoken_num - 1)
          data['sentence_map'].append(sent_num)
          data['subtoken_map'].append(subtoken_num)

      ctoken = raw_tokens[0]
      cpos = 0
      for token in tokens:
          data['sentences'][-1].append(token)
          data['speakers'][-1].append("-")
          data['sentence_map'].append(sent_num)
          data['subtoken_map'].append(subtoken_num)
          
          if token.startswith("##"):
              token = token[2:]
          if len(ctoken) == len(token):
              subtoken_num += 1
              cpos += 1
              if cpos < len(raw_tokens):
                  ctoken = raw_tokens[cpos]
          else:
              ctoken = ctoken[len(token):]
      sent_n = sent_num

  data['sentences'][-1].append("[SEP]")
  data['speakers'][-1].append("[SPL]")
  data['sentence_map'].append(sent_n - 1)
  data['subtoken_map'].append(subtoken_num - 1)

  with open("sample.in.json", 'w') as out:
      json.dump(data, out, sort_keys=True)

data_convert(text)

! cat sample.in.json

{"clusters": [], "doc_key": "nw", "sentence_map": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

Get Prediction

In [ ]:
! GPU=0 python predict.py $CHOSEN_MODEL sample.in.json sample.out.txt

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

# Output Handling

Finally, we do a little processing to get the output to have the same token indices as our input.

In [ ]:
def convert_mention(mention,output,comb_text):
    start = output['subtoken_map'][mention[0]]
    end = output['subtoken_map'][mention[1]] + 1
    nmention = (start, end)
    mtext = ''.join(' '.join(comb_text[mention[0]:mention[1]+1]).split(" ##"))
    return (nmention, mtext)

In [ ]:
def out_put(mention_l,convert_mention):
  output = json.load(open("sample.out.txt"))

  comb_text = [word for sentence in output['sentences'] for word in sentence]


  seen = set()
  print('Clusters:')
  for cluster in output['predicted_clusters']:
      mapped = []
      for mention in cluster:
          seen.add(tuple(mention))
          mapped.append(convert_mention(mention,output,comb_text))
      print(mapped, end=",\n")

  print('\nMentions:')
  for mention in output['top_spans']:
      if tuple(mention) in seen:
          continue
      mention_l.append(convert_mention(mention,output,comb_text))
      print(convert_mention(mention,output,comb_text), end=",\n")
  return mapped , mention_l

In [ ]:
text[0]

'Stereo scene capture and generation is an important facet of presence research in that stereoscopic images have been linked to naturalness as a component of reported presence. Three-dimensional images can be captured and presented in many ways, but it is rare that the most simple and “natural” method is used: full orthostereoscopic image capture and projection. This technique mimics as closely as possible the geometry of the human visual system and uses convergent axis stereography with the cameras separated by the human interocular distance. It simulates human viewing angles, magnification, and convergences so that the point of zero disparity in the captured scene is reproduced without disparity in the display. In a series of experiments, we have used this technique to investigate body image distortion in photographic images. Three psychophysical experiments compared size, weight, or shape estimations (perceived waist-hip ratio) in 2-D and 3-D images for the human form and real or vi

In [ ]:
mention_l = []
clusters,mentions = out_put(mention_l,convert_mention)

Clusters:
[((42, 48), 'the most simple and “ natural ” method'), ((56, 58), 'This technique'), ((84, 85), 'It'), ((118, 120), 'this technique'), ((247, 249), 'The technique')],
[((212, 215), 'the visual system'), ((217, 218), 'it')],
[((219, 220), 'generate'), ((232, 233), 'This')],
[((252, 264), 'more - accurate estimations of object shape or size and control of ocular suppression'), ((264, 265), 'These')],

Mentions:
((0, 1), '[CLS]'),
((0, 5), 'Stereo scene capture and generation'),
((5, 6), 'is'),
((10, 12), 'presence research'),
((14, 15), 'stereoscopic'),
((14, 16), 'stereoscopic images'),
((18, 19), 'linked'),
((20, 27), 'naturalness as a component of reported presence'),
((20, 27), 'naturalness as a component of reported presence .'),
((20, 37), 'naturalness as a component of reported presence . Three - dimensional images can be captured and presented in many ways'),
((20, 37), 'naturalness as a component of reported presence . Three - dimensional images can be captured and pre